In [1]:
import h5py
%matplotlib inline
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import sys
import shutil
import glob
import optparse
import os
import json
import pandas as pd
import numpy as np
import pylab as pl
import scipy.stats as stats
import seaborn as sns
sns.set_style("ticks")
sns.set()
sns.set_color_codes()

def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

def normalize_across_rows(in_array):

    min_val = np.min(in_array,0)
    max_val = np.max(in_array,0)

    min_mat = np.tile(min_val,(in_array.shape[0],1))
    max_mat = np.tile(max_val,(in_array.shape[0],1))
    range_mat = max_mat - min_mat

    out_array = np.true_divide(in_array-min_mat,range_mat)

    out_array[np.isnan(out_array)] = 0
    
    return out_array

def normalize_across_rows2(in_array):

    min_val = np.min(in_array,0)
    max_val = np.max(in_array,0)

    min_mat = np.tile(min_val,(in_array.shape[0],1))
    max_mat = np.tile(max_val,(in_array.shape[0],1))
    range_mat = max_mat - min_mat

    out_array = np.true_divide(in_array,max_mat)

    out_array[np.isnan(out_array)] = 0
    
    return out_array

def get_upper_triangle_values(in_array):
    mask = np.ones(in_array.shape)
    tmp = np.tril(mask,0)
    out_val = in_array[np.where(tmp==0)]
    
    return out_val

In [2]:


dset_list = ['V1_JC085_20190624','V1_JC085_20190712','V1_JC097_20190621','V1_JC097_20190628',\
             'V1_JC097_20190704',\
             'LM_JC080_20190619','LM_JC091_20190621','LM_JC091_20190703','LM_JC091_20190628',\
             'LM_JC097_20190702','LM_JC097_20190708','LM_JC085_20190704',\
             'LI_JC091_20190625','LI_JC091_20190701','LI_JC091_20190705'
            ]


response_type = 'df_f'

filter_crit = 'zscore'
filter_thresh = 1

#define paths
aggregate_root = '/n/coxfs01/cechavarria/2p-aggregate/scenes'
fig_base_dir = os.path.join(aggregate_root,'RSA_pooled','figures','%s_%i'%(filter_crit, filter_thresh))


In [20]:
fig_base_dir

'/n/coxfs01/cechavarria/2p-aggregate/scenes/RSA_pooled/figures/zscore_1'

In [5]:
#get responses
area_list = ['V1','LM','LI']
animalid_list = []
sess_count = []
areaid = np.zeros((len(dset_list,)))
animalid = np.zeros((len(dset_list,)))
sessid = np.zeros((len(dset_list,)))

#consider all cells
V1_response_all = []
LM_response_all = []
LI_response_all = []

#consider only responsive cells
V1_response_active = []
LM_response_active = []
LI_response_active = []



for dset_idx, dset in enumerate(dset_list):
# dset_idx = 0
# dset = dset_list[dset_idx]

    #figure out some indexes
    i0 = findOccurrences(dset,'_')[0]
    i1 = findOccurrences(dset,'_')[1]

    area = dset[0:i0]
    animal = dset[i0+1:i1]
    session = dset[i1+1:]

    if animal not in animalid_list:
        animalid_list.append(animal)
        sess_count.append(0)

    sess_count[animalid_list.index(animal)] = sess_count[animalid_list.index(animal)]+1

    areaid[dset_idx] = area_list.index(area)
    animalid[dset_idx] = animalid_list.index(animal)
    sessid[dset_idx] = sess_count[animalid_list.index(animal)]-1


    #load data
    aggregate_file_dir = os.path.join(aggregate_root,area,'files','filtered_responses')
    data_array_fn = '%s_%s_filtered_%s_responses_thresh_%s_%i.hdf5'%(animal, session, response_type, filter_crit, filter_thresh)
    data_array_filepath = os.path.join(aggregate_file_dir, data_array_fn)
    data_grp = h5py.File(data_array_filepath, 'r')

    config_cond = np.array(data_grp['config_cond'])
    curr_slice = 'Slice01'#hard-code planar data for now
    
    active_cell_idx = np.array(data_grp['/'.join([curr_slice, 'active_cell_idx'])])
    #active_cell_idx marks cells with responses to at least one condition
    dset_response = np.array(data_grp['/'.join([curr_slice, 'filtered_response_per_cfg_per_neuron'])])
    dset_response_active = dset_response[:,active_cell_idx]

    if area == 'V1':
        if len(V1_response_active)==0:
            V1_response_active = dset_response_active
            V1_response_all = dset_response
        else:
            V1_response_active = np.hstack((V1_response_active, dset_response_active))
            V1_response_all = np.hstack((V1_response_all, dset_response))
    elif area == 'LM':
        if len(LM_response_active)==0:
            LM_response_active = dset_response_active
            LM_response_all = dset_response
        else:
            LM_response_active = np.hstack((LM_response_active, dset_response_active))
            LM_response_all = np.hstack((LM_response_all, dset_response))
    elif area == 'LI':
        if len(LI_response_active)==0:
            LI_response_active = dset_response_active
            LI_response_all = dset_response
        else:
            LI_response_active = np.hstack((LI_response_active, dset_response_active))
            LI_response_all = np.hstack((LI_response_all, dset_response))

In [6]:
#replace nans with 0's
V1_response_active[np.isnan(V1_response_active)] = 0
V1_response_all[np.isnan(V1_response_all)] = 0

LM_response_active[np.isnan(LM_response_active)] = 0
LM_response_all[np.isnan(LM_response_all)] = 0

LI_response_active[np.isnan(LI_response_active)] = 0
LI_response_all[np.isnan(LI_response_all)] = 0

#run correlations
R_V1_all = np.corrcoef(V1_response_all)
R_V1_active = np.corrcoef(V1_response_active)

R_LM_all = np.corrcoef(LM_response_all)
R_LM_active = np.corrcoef(LM_response_active)

R_LI_all = np.corrcoef(LI_response_all)
R_LI_active = np.corrcoef(LI_response_active)

In [7]:
#plot individual area correlation matrices
fig_out_dir = os.path.join(fig_base_dir,'raw_%s'%response_type)
if not os.path.isdir(fig_out_dir):
        os.makedirs(fig_out_dir)

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_V1_all,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_V1_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_V1_active,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_V1_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LM_all,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_LM_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LM_active,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_LM_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LI_all,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_LI_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LI_active,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_LI_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

#compare correlation matrices across areas
R_V1_all_val = get_upper_triangle_values(R_V1_all)
R_LM_all_val = get_upper_triangle_values(R_LM_all)
R_LI_all_val = get_upper_triangle_values(R_LI_all)

R_V1_active_val = get_upper_triangle_values(R_V1_active)
R_LM_active_val = get_upper_triangle_values(R_LM_active)
R_LI_active_val = get_upper_triangle_values(R_LI_active)

R_all_areas = np.vstack((R_V1_all_val, R_LM_all_val, R_LI_all_val))
R_across_areas_all = np.corrcoef(R_all_areas)


R_active_areas = np.vstack((R_V1_active_val, R_LM_active_val, R_LI_active_val))
R_across_areas_active = np.corrcoef(R_active_areas)



#plot cross-area correlations
plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_across_areas_all,center = 0,yticklabels = area_list,xticklabels = area_list,\
                 annot=True, cmap = 'RdBu_r')

fig_name = 'across_areas_all_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_across_areas_active,center = 0,yticklabels = area_list,xticklabels = area_list,\
                 annot=True, cmap = 'RdBu_r')

fig_name = 'across_areas_active_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

In [8]:
#re-arrange by stimulus class
V1_response_all_class = np.vstack((V1_response_all[0:60:3,:],V1_response_all[1:60:3,:],V1_response_all[2:60:3,:]))
V1_response_active_class = np.vstack((V1_response_active[0:60:3,:],V1_response_active[2:60:3,:],V1_response_active[2:60:3,:]))


LM_response_all_class = np.vstack((LM_response_all[0:60:3,:],LM_response_all[1:60:3,:],LM_response_all[2:60:3,:]))
LM_response_active_class = np.vstack((LM_response_active[0:60:3,:],LM_response_active[1:60:3,:],LM_response_active[2:60:3,:]))


LI_response_all_class = np.vstack((LI_response_all[0:60:3,:],LI_response_all[1:60:3,:],LI_response_all[2:60:3,:]))
LI_response_active_class = np.vstack((LI_response_active[0:60:3,:],LI_response_active[1:60:3,:],LI_response_active[2:60:3,:]))




#run correlations
R_V1_all_class = np.corrcoef(V1_response_all_class)
R_V1_active_class = np.corrcoef(V1_response_active_class)

R_LM_all_class = np.corrcoef(LM_response_all_class)
R_LM_active_class = np.corrcoef(LM_response_active_class)

R_LI_all_class = np.corrcoef(LI_response_all_class)
R_LI_active_class = np.corrcoef(LI_response_active_class)

#plot individual area correlation matrices
fig_out_dir = os.path.join(fig_base_dir,'raw_%s'%response_type)
if not os.path.isdir(fig_out_dir):
        os.makedirs(fig_out_dir)

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_V1_all_class,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_class_V1_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_V1_active_class,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_class_V1_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LM_all_class,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_class_LM_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LM_active_class,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_class_LM_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LI_all_class,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_class_LI_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LI_active_class,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_class_LI_raw_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

In [9]:
#zscore data across configs
V1_response_all_zscore = stats.zscore(V1_response_all,0)
V1_response_active_zscore = stats.zscore(V1_response_active,0)

LM_response_all_zscore = stats.zscore(LM_response_all,0)
LM_response_active_zscore = stats.zscore(LM_response_active,0)

LI_response_all_zscore = stats.zscore(LI_response_all,0)
LI_response_active_zscore = stats.zscore(LI_response_active,0)


#replace nans with 0's
V1_response_all_zscore[np.isnan(V1_response_all_zscore)] = 0
V1_response_active_zscore[np.isnan(V1_response_active_zscore)] = 0

LM_response_all_zscore[np.isnan(LM_response_all_zscore)] = 0
LM_response_active_zscore[np.isnan(LM_response_active_zscore)] = 0

LI_response_all_zscore[np.isnan(LI_response_all_zscore)] = 0
LI_response_active_zscore[np.isnan(LI_response_active_zscore)] = 0

/n/coxfs01/cechavarria/envs/suite2p/lib/python3.6/site-packages/scipy/stats/stats.py:2281: RuntimeWarning: divide by zero encountered in true_divide
  return (a - mns) / sstd
/n/coxfs01/cechavarria/envs/suite2p/lib/python3.6/site-packages/scipy/stats/stats.py:2281: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [10]:
#get correlations
R_V1_all_zscore = np.corrcoef(V1_response_all_zscore)
R_V1_active_zscore = np.corrcoef(V1_response_active_zscore)

R_LM_all_zscore = np.corrcoef(LM_response_all_zscore)
R_LM_active_zscore = np.corrcoef(LM_response_active_zscore)

R_LI_all_zscore = np.corrcoef(LI_response_all_zscore)
R_LI_active_zscore = np.corrcoef(LI_response_active_zscore)

In [11]:
#plot individual area correlation matrices
fig_out_dir = os.path.join(fig_base_dir,'zscore_%s'%response_type)
if not os.path.isdir(fig_out_dir):
        os.makedirs(fig_out_dir)

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_V1_all_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_V1_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)


fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_V1_active_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_V1_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LM_all_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_LM_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LM_active_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_LM_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LI_all_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_LI_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LI_active_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_LI_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

In [12]:
#compare correlation matrices across areas
R_V1_all_zscore_val = get_upper_triangle_values(R_V1_all_zscore)
R_LM_all_zscore_val = get_upper_triangle_values(R_LM_all_zscore)
R_LI_all_zscore_val = get_upper_triangle_values(R_LI_all_zscore)

R_V1_active_zscore_val = get_upper_triangle_values(R_V1_active_zscore)
R_LM_active_zscore_val = get_upper_triangle_values(R_LM_active_zscore)
R_LI_active_zscore_val = get_upper_triangle_values(R_LI_active_zscore)

R_all_areas_zscore = np.vstack((R_V1_all_zscore_val, R_LM_all_zscore_val, R_LI_all_zscore_val))
R_across_areas_all_zscore = np.corrcoef(R_all_areas_zscore)

R_active_areas_zscore = np.vstack((R_V1_active_zscore_val, R_LM_active_zscore_val, R_LI_active_zscore_val))
R_across_areas_active_zscore = np.corrcoef(R_active_areas_zscore)


#plot cross-area correlations
plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_across_areas_all_zscore,center = 0,yticklabels = area_list,xticklabels = area_list,\
                 annot=True, cmap = 'RdBu_r')

fig_name = 'across_areas_all_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_across_areas_active_zscore,center = 0,yticklabels = area_list,xticklabels = area_list,\
                 annot=True, cmap = 'RdBu_r')

fig_name = 'across_areas_active_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

In [13]:
#zscore data across configs for re-arranged matrices
V1_response_all_class_zscore = stats.zscore(V1_response_all_class,0)
V1_response_active_class_zscore = stats.zscore(V1_response_active_class,0)

LM_response_all_class_zscore = stats.zscore(LM_response_all_class,0)
LM_response_active_class_zscore = stats.zscore(LM_response_active_class,0)

LI_response_all_class_zscore = stats.zscore(LI_response_all_class,0)
LI_response_active_class_zscore = stats.zscore(LI_response_active_class,0)



#replace nans with 0's
V1_response_all_class_zscore[np.isnan(V1_response_all_class_zscore)] = 0
V1_response_active_class_zscore[np.isnan(V1_response_active_class_zscore)] = 0

LM_response_all_class_zscore[np.isnan(LM_response_all_class_zscore)] = 0
LM_response_active_class_zscore[np.isnan(LM_response_active_class_zscore)] = 0

LI_response_all_class_zscore[np.isnan(LI_response_all_class_zscore)] = 0
LI_response_active_class_zscore[np.isnan(LI_response_active_class_zscore)] = 0

In [14]:
#get correlations
R_V1_all_class_zscore = np.corrcoef(V1_response_all_class_zscore)
R_V1_active_class_zscore = np.corrcoef(V1_response_active_class_zscore)

R_LM_all_class_zscore = np.corrcoef(LM_response_all_class_zscore)
R_LM_active_class_zscore = np.corrcoef(LM_response_active_class_zscore)

R_LI_all_class_zscore = np.corrcoef(LI_response_all_class_zscore)
R_LI_active_class_zscore = np.corrcoef(LI_response_active_class_zscore)

In [15]:

#plot individual area correlation matrices
fig_out_dir = os.path.join(fig_base_dir,'zscore_%s'%response_type)
if not os.path.isdir(fig_out_dir):
        os.makedirs(fig_out_dir)

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_V1_all_class_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_class_V1_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)


fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_V1_active_class_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_class_V1_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LM_all_class_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_class_LM_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LM_active_class_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_class_LM_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LI_all_class_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_class_LI_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LI_active_class_zscore,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_class_LI_zscore_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

In [16]:
#perform min-max normlization across configs
V1_response_all_norm = normalize_across_rows(V1_response_all)
V1_response_active_norm = normalize_across_rows(V1_response_active)

LM_response_all_norm = normalize_across_rows(LM_response_all)
LM_response_active_norm = normalize_across_rows(LM_response_active)

LI_response_all_norm = normalize_across_rows(LI_response_all)
LI_response_active_norm = normalize_across_rows(LI_response_active)

/n/coxfs01/cechavarria/envs/suite2p/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in true_divide
/n/coxfs01/cechavarria/envs/suite2p/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide


In [17]:
#perform correlation
R_V1_all_norm = np.corrcoef(V1_response_all_norm)
R_V1_active_norm = np.corrcoef(V1_response_active_norm)

R_LM_all_norm = np.corrcoef(LM_response_all_norm)
R_LM_active_norm = np.corrcoef(LM_response_active_norm)

R_LI_all_norm = np.corrcoef(LI_response_all_norm)
R_LI_active_norm = np.corrcoef(LI_response_active_norm)

In [18]:
#plot individual area correlation matrices
fig_out_dir = os.path.join(fig_base_dir,'norm_%s'%response_type)
if not os.path.isdir(fig_out_dir):
        os.makedirs(fig_out_dir)

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_V1_all_norm,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_V1_norm_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)


fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_V1_active_norm,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_V1_norm_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LM_all_norm,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_LM_norm_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LM_active_norm,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_LM_norm_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LI_all_norm,center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'all_LI_norm_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_LI_active_norm, center = 0,annot=False, cmap = 'RdBu_r')

fig_name = 'active_LI_norm_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

In [19]:
#compare correlation matrices across areas
R_V1_all_norm_val = get_upper_triangle_values(R_V1_all_norm)
R_LM_all_norm_val = get_upper_triangle_values(R_LM_all_norm)
R_LI_all_norm_val = get_upper_triangle_values(R_LI_all_norm)

R_V1_active_norm_val = get_upper_triangle_values(R_V1_active_norm)
R_LM_active_norm_val = get_upper_triangle_values(R_LM_active_norm)
R_LI_active_norm_val = get_upper_triangle_values(R_LI_active_norm)

R_all_areas_norm = np.vstack((R_V1_all_norm_val, R_LM_all_norm_val, R_LI_all_norm_val))
R_across_areas_all_norm = np.corrcoef(R_all_areas_norm)

R_active_areas_norm = np.vstack((R_V1_active_norm_val, R_LM_active_norm_val, R_LI_active_norm_val))
R_across_areas_active_norm = np.corrcoef(R_active_areas_norm)


#plot cross-area correlations
plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_across_areas_all_norm,center = 0,yticklabels = area_list,xticklabels = area_list,\
                 annot=True, cmap = 'RdBu_r')

fig_name = 'across_areas_all_norm_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(R_across_areas_active_norm,center = 0,yticklabels = area_list,xticklabels = area_list,\
                 annot=True, cmap = 'RdBu_r')

fig_name = 'across_areas_active_norm_pool_%s_RSA.png'%(response_type)
fig_fn = os.path.join(fig_out_dir,fig_name)

fig = ax.get_figure()
fig.savefig(fig_fn) 
plt.close()

In [ ]:
# 

In [ ]:
#----Junk Code below here------